# Group: CopyPaste
#### Dataset used: Dataset_Hospital_Vists.csv, test.csv
#### Participants: Ligia, Gina, Raza

### Contents Draft

1. Problem and Goal Definition
2. Data Understanding
3. Data Quality Check
4. Exploratory Data Analysis
5. Data Cleaning and Preprocessing
6. Train-Test Split
7. Model Building
8. Model Evaluation
9. Model Interpretation
10. Fine-Tuning and Optimization

### 1. Problem and Goal Definition
Problem: Patients in a hospital miss their scheduled appointments.

Goal: Develop a machine learning model that predicts if a patient will miss a future appointment.

### 2. Data Understanding
#### 2.1 Dataset Description

- The dataset is at appointment level granularity and contains detail of each appointment and patient.

- It has 14 columns of which 1 will be our target variable: No-show.

- We mostly have information about an appointment's date and place and the patients' health details. A column also shows if a patient received an SMS before the appointment.

#### 2.2 Quick Analysis from Kaggle

- There is missing data in the columns Age, Community, Social Welfare, and some diseases.
- Female to Male ratio is 65:35.
- We have no NULLs in the target variable.
- For Handcap, we have multiple values even though it seems to be a binary variable.

### 3. Data Quality Check

- Uniqueness
- Missing data
- Data type consistency check
- Distribution of Categorical Variables
- Dates inconsistency

In [ ]:
# Setting up environment with packages

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
# Importing the dataset

df = pd.read_csv("/Users/muhammadraza/Documents/GitHub/BIPM/Data Science/Project/Dataset_Hospital_Visits.csv")
df.head()

# Increase seaborn default resolution
sns.set(rc={"figure.dpi":150, 'savefig.dpi':150})
sns.set_context('notebook')
sns.set_style("ticks")
sns.set(rc={'figure.figsize':(5,6)})

# Give variables to color numbers

green = '#008000'
red = '#ff0000'


In [ ]:
## Uniqueness

# Is each row unique?

print("Duplicate rows: " + str(df.duplicated().sum()))

# Is each appointmentID unique?

print("Duplicate appointments: " + str(df['AppointmentID'].duplicated().sum()))

We can conclude that the dataset only containts unique IDs and no duplicates.

In [ ]:
## Missing Data

# Which columns have missing data?

missing_data = df.isnull().sum()
total_entries = len(df)
percentage_missing = round((missing_data / total_entries) * 100, 2)

missing_info = pd.DataFrame({
    'Missing Count': missing_data,
    'Percentage Missing': percentage_missing
})

print(missing_info)

As seen from Kaggle, Age, Community, SocialWelfare, Hipertension, and Alcoholism have significant null values.

In [ ]:
## Data Type Consistency

df.dtypes

1. PatientId and Age are to be converted to int from float as IDs must be int/str and ages are usually considered in whole number terms.
2. ScheduledDate and AppointmentDate must be timestamps and not objects.

In [ ]:
# Distribution of Categorical Variables

occ = df.groupby('Handcap').size().reset_index()
print(occ)

We assume that this column was meant to be a binary column and the numerical values are bad data. They will be removed during data cleaning.

In [ ]:
# Dates inconsistency

from datetime import datetime

df['AppointmentDate'] = df['AppointmentDate'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%SZ').date())
df['ScheduledDate'] = df['ScheduledDate'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%SZ').date())

counts = df['ScheduledDate'] > df['AppointmentDate']
occurrence_counts = counts.value_counts()

print(occurrence_counts)

We see that we have 4 incoherent combination of schedule and appointment dates - they will be taken out in data cleaning stage.

### 4. Exploratory Data Analysis

- Distribution of the target variable.
- Distribution of age. Which age groups account for the most missing appointments?
- Do patients of a certain community miss their appointments more than others?
- Do males or females miss more appointments?

In [ ]:
## Distribution of the target variable

value_counts = df['No-show'].value_counts()

labels = value_counts.index
sizes = value_counts.values

colors = [green,red]  # Customize colors
explode = (0.05, 0)  # Explode the 1st slice

plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90, explode=explode)

plt.title('Distribution of No-Show Variable')
plt.axis('equal')  # Equal aspect ratio ensures the pie chart is circular

plt.show()

1 in 5 appointments are missed on average.

In [ ]:
# Which age group misses more appointments?

# Plot histogram with split bars

plt.hist([df[df['No-show'] == 'No']['Age'], df[df['No-show'] == 'Yes']['Age']],
         bins=10, color=['green', 'red'], alpha=0.7, edgecolor='black', label=['No', 'Yes'], stacked=True)

plt.title('Overall Age Distribution')
plt.xlabel('Age')
plt.ylabel('Count')
plt.legend()

# To add % later:

# n, bins, _ = plt.hist([df[df['No-show'] == 'No']['Age'], df[df['No-show'] == 'Yes']['Age']],
#          bins=10, color=['green', 'red'], alpha=0.7, edgecolor='black', label=['No', 'Yes'], stacked=True)

# for i in range(len(bins) - 1):
#     total = n[0][i] + n[1][i]
#     plt.text((bins[i] + bins[i + 1]) / 2, total, f'{n[0][i] / total:.0%}', ha='center', va='bottom')
#     plt.text((bins[i] + bins[i + 1]) / 2, total, f'{n[1][i] / total:.0%}', ha='center', va='top')

plt.show()

There is an even distribution of the ratio of appointments missed in the various age groups. This tends to change after age 70 where appointments are missed a lot less.

This could be explained by the fact that older people cannot afford to miss appointments due to more serious health issues and due to the fact that they might have more time on their hand.

In [ ]:
## Which gender misses more appointments?

# Group by 'gender' and 'no_show' and count occurrences
grouped_data = df.groupby(['Sex', 'No-show']).size().unstack()

# Calculate percentages
percentages = grouped_data.div(grouped_data.sum(axis=1), axis=0) * 100

# Plotting a grouped bar chart
ax = percentages.plot(kind='bar', stacked=True, color=[green, red])

# Annotate bars with percentages
for p in ax.patches:
    width, height = p.get_width(), p.get_height()
    x, y = p.get_xy() 
    ax.annotate(f'{height:.1f}%', (x + width/2, y + height/2), ha='center', va='center')

plt.title('Gender-wise No-show Distribution')
plt.xlabel('Sex')
plt.ylabel('# Appointments')
plt.legend(title='No-show', loc='upper right')

plt.show()

There seems to be 1 in 5 appointments missed for both genders.

In [ ]:

# Do patients of a certain community miss their appointments more than others?

# Count the total number of appointments per community
total_appointments_per_community = df['Community'].value_counts()

# Sort the DataFrame based on the total number of appointments
sorted_df = df[df['Community'].isin(total_appointments_per_community.index)].copy()
sorted_df['Community'] = pd.Categorical(sorted_df['Community'], categories=total_appointments_per_community.index, ordered=True)
sorted_df = sorted_df.sort_values(by=['Community'])

# Count the number of appointments per community split by show_up status
appointments_per_community_show_up = sorted_df.groupby(['Community', 'No-show']).size().unstack(fill_value=0)

# Plotting the bar chart
fig, ax = plt.subplots(figsize=(10, 6))

# Bar chart for total appointments per community
total_appointments_per_community.loc[sorted_df['Community'].unique()].plot(kind='bar', ax=ax, color='blue', label='Total Appointments')

# Bar chart for appointments per community split by show_up status
appointments_per_community_show_up.plot(kind='bar', stacked=True, ax=ax, color=['green', 'red'], label=['Show Up', 'No Show Up'])

# Adding labels and legend
ax.set_title('Appointments per Community')
ax.set_xlabel('Community')
ax.set_ylabel('Number of Appointments')
ax.legend()

plt.show()

### 5. Data Cleaning and Preprocessing (incl. additional EDA)

- Dropping erroneous data
- Updating Existing Features: Get the columns into the right data types (has some errors but delayed due to non-urgency)
- Updating Existing Features: Handling Missing Data: Extrapolation + Imputation (?)
- Adding New Features: Time between ScheduledDate and AppointmentDate
- Standardization of Continuous variable: yes or no?
- OneHotEncoding for Categorical Vairables
- Feature Selection based on Correlation Matrix
- Feature Selection based on Information Gain
- Feature Selection based on Automated Methods i.e. SelectKBest()

In [ ]:
# Keeping original df intact:

df_t = df[:]

In [ ]:
df_t.info()

In [ ]:
## Data Type Consistency

df_t.dtypes

In [ ]:
# Dropping erroneous data

# Dropping inconsistent dates from transformed dataframe

df_t = df_t.drop(df[df['ScheduledDate'] > df['AppointmentDate']].index)


In [ ]:
# Dropping outliers for Handcap: 2, 3, 4

df_t['Handcap'] = df_t['Handcap'][df_t['Handcap'].isin(['yes', 'no'])]

In [ ]:
df_t.info()

In [ ]:
occ = df_t.groupby('Handcap').size().reset_index()
print(occ)

In [ ]:
# PatientId and Age are to be converted to int from float as IDs must be int/str and ages are usually considered in whole number terms.

# Has an error!!!

# df_t["PatientId"] = pd.to_numeric(df["PatientId"], errors='coerce').astype(int)
# df_t["Age"] = pd.to_numeric(df["Age"], errors='coerce').astype(int)


In [ ]:
# Extrapolating missing data

missing_columns = ['Age', 'Community', 'SocialWelfare', 'Hipertension', 'Alcoholism']

for column in missing_columns:
    df_t[column] = df_t.groupby('PatientId')[column].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))


In [ ]:
missing_data = df_t.isnull().sum()
total_entries = len(df_t)
percentage_missing = round((missing_data / total_entries) * 100, 2)

missing_info = pd.DataFrame({
    'Missing Count': missing_data,
    'Percentage Missing': percentage_missing
})

print(missing_info)

In [ ]:
## Dropping missing values for time being. In the future we could also use the KNN imputer to fill empty values.

df_t.dropna(inplace=True)

In [ ]:
df_t.info()

In [ ]:
# Apprx loss of data

(1- (df_t.count()/ df.count())) * 100

In [ ]:
df_t.dtypes

In [ ]:
# Adding New Features: Time between ScheduledDate and AppointmentDate

from datetime import timedelta

df_t['time_bw_schedule_appointment'] = df_t['AppointmentDate'] - df_t['ScheduledDate']

## Convert to float (days)

df_t['time_bw_schedule_appointment'] = df_t['time_bw_schedule_appointment'] / pd.Timedelta(days=1)
df_t['time_bw_schedule_appointment'] = df_t['time_bw_schedule_appointment'].astype(float)



In [ ]:
# Plot histogram with split bars ## EDA and can be removed later.

plt.hist([df_t[df_t['No-show'] == 'No']['time_bw_schedule_appointment'], df_t[df_t['No-show'] == 'Yes']['time_bw_schedule_appointment']],
         bins=10, color=['green', 'red'], alpha=0.7, edgecolor='black', label=['No', 'Yes'], stacked=True)

plt.title('Time Diff Distribution')
plt.xlabel('Days')
plt.ylabel('Count')
plt.legend()

plt.show()

In [ ]:
# Take relevant columns for correlation matrix

df_sub = df_t[['Diabetes', 'Sex', 'Alcoholism', 'Hipertension', 'SMS_received', 'Handcap', 'time_bw_schedule_appointment', 'No-show']]

In [ ]:
# Feature Selection: Correlation Matrix to understand which features have highest correlation with target variable and less correlation amongst each other.

df_t_encoded = pd.get_dummies(df_sub, columns=['Diabetes', 'Sex', 'Alcoholism', 'Hipertension', 'SMS_received', 'Handcap', 'No-show'], drop_first=True).astype(int)
correlation_matrix = df_t_encoded.corr()

sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=.5)

In [ ]:
# Feature Selection based on Info Gain

from sklearn.feature_selection import mutual_info_classif

X = df_t_encoded.drop("No-show_Yes", axis=1)
y = df_t_encoded['No-show_Yes']

importances = mutual_info_classif(X, y)
feature_importances = pd.Series(importances, df_t_encoded.columns[0:len(df_t_encoded.columns)-1])
feature_importances.plot(kind="bar", color="teal")
plt.show()

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_regression, f_classif
from numpy import array

# Create training set and prediction target
X = df_t_encoded.drop("No-show_Yes", axis=1)
y = df_t_encoded['No-show_Yes']

# Perform feature selection
# Set k to the number of features you want to identify
select = SelectKBest(score_func=f_classif, k=4)
select.fit_transform(X,y)

# Print feature names
filter = select.get_support() 
features = array(X.columns)
 
print("Selected best:")
print(features[filter])

In [ ]:
df_t.info()

In [ ]:
df_t = df_t.applymap(lambda x: x.capitalize() if isinstance(x, str) else x)

### 6. Train-Test Split

In [ ]:
from sklearn.model_selection import train_test_split

X = df_t.drop(columns=['No-show', 'PatientId', 'AppointmentID', 'ScheduledDate', 'AppointmentDate', 'Community'])
y = df_t['No-show']

from sklearn.preprocessing import LabelEncoder

# Create an instance of LabelEncoder
label_encoder = LabelEncoder()

y = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
df_t.info()

In [ ]:
# One Hot Encoding of Categorical Variables

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse_output=False, drop='if_binary')

ct = ColumnTransformer(
    [('OneHotEncoder', ohe, ['SocialWelfare', 'Diabetes', 'Sex', 'Alcoholism', 'Hipertension', 'SMS_received', 'Handcap'])],
    remainder='passthrough'
)

ct.fit_transform(X_train)

In [ ]:
x = ct.fit_transform(X_train)

df_X = pd.DataFrame(x)

print(df_X)

### 7. Model Building


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline

clf = DecisionTreeClassifier()

# Create pipeline
pipe = Pipeline([
    ('preprocessor', ct),
    ('classifier', clf)]
)

In [ ]:
pipe

In [ ]:
pipe.fit(X_train, y_train)

In [ ]:
y_pred = pipe.predict(X_test)

### 8. Model Evaluation

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

display_labels=['No-Show', 'Show']
cm = confusion_matrix(y_test, y_pred, labels=clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                               display_labels=display_labels)
disp.plot()
plt.show()

In [ ]:
# Print a classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

## This is the processing for test.

In [ ]:
# Keeping original df intact:

df_test = pd.read_csv('/Users/muhammadraza/Documents/GitHub/BIPM/Data Science/Project/test.csv')

df_test_t = df_test[:]

In [ ]:
df_test_t.info()

In [ ]:
## Data Type Consistency

df_test_t.dtypes

In [ ]:
# Dropping erroneous data

# No need to change dates.

In [ ]:
# Dropping outliers for Handcap: 2, 3, 4

import numpy as np

df_test_t['Handcap'] = np.where(df_test_t['Handcap'] != 'yes', 'no', df_test_t['Handcap'])

In [ ]:
df_test_t.info()

In [ ]:
# Extrapolating missing data

missing_columns = ['Age', 'Community', 'SocialWelfare', 'Hipertension', 'Alcoholism']

for column in missing_columns:
    df_test_t[column] = df_test_t.groupby('PatientId')[column].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))


In [ ]:
df_test_t.info()

In [ ]:
# Adding New Features: Time between ScheduledDate and AppointmentDate

from datetime import timedelta

from datetime import datetime

df_test_t['AppointmentDate'] = df_test_t['AppointmentDate'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%SZ').date())
df_test_t['ScheduledDate'] = df_test_t['ScheduledDate'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%SZ').date())


In [ ]:

df_test_t['time_bw_schedule_appointment'] = df_test_t['AppointmentDate'] - df_test_t['ScheduledDate']

## Convert to float (days)

df_test_t['time_bw_schedule_appointment'] = df_test_t['time_bw_schedule_appointment'] / pd.Timedelta(days=1)
df_test_t['time_bw_schedule_appointment'] = df_test_t['time_bw_schedule_appointment'].astype(float)



In [ ]:
# Take relevant columns for correlation matrix

df_sub_test = df_test_t[['Diabetes', 'Sex', 'Alcoholism', 'Hipertension', 'SMS_received', 'Handcap', 'time_bw_schedule_appointment']]

In [ ]:
df_test_t.info()

In [ ]:
df_test_t = df_test_t.applymap(lambda x: x.capitalize() if isinstance(x, str) else x)

In [ ]:
X_df_test = df_test_t.drop(columns=['PatientId', 'AppointmentID', 'ScheduledDate', 'AppointmentDate', 'Community'])

X_df_test.info()

In [320]:
from sklearn.impute import SimpleImputer

imputer_cont = SimpleImputer(strategy="median", add_indicator=False)
imputer_cat = SimpleImputer(strategy="most_frequent", add_indicator=False)

ct_test = ColumnTransformer(
    [('impute_age', imputer_cont, ['Age']),
    ('impute_cat', imputer_cat, ['SocialWelfare', 'Diabetes','Alcoholism', 'Hipertension']),
    ('OneHotEncoder', ohe, ['SocialWelfare', 'Diabetes', 'Sex', 'Alcoholism', 'Hipertension', 'SMS_received', 'Handcap'])],
    remainder='passthrough'
)

ct_test.fit_transform(X_df_test)

ct_test.get_feature_names_out()

## Has 9 columns as expected

array(['impute_age__Age', 'impute_cat__SocialWelfare',
       'impute_cat__Diabetes', 'impute_cat__Alcoholism',
       'impute_cat__Hipertension', 'OneHotEncoder__SocialWelfare_Yes',
       'OneHotEncoder__SocialWelfare_nan', 'OneHotEncoder__Diabetes_Yes',
       'OneHotEncoder__Sex_M', 'OneHotEncoder__Alcoholism_Yes',
       'OneHotEncoder__Alcoholism_nan', 'OneHotEncoder__Hipertension_Yes',
       'OneHotEncoder__Hipertension_nan',
       'OneHotEncoder__SMS_received_Yes', 'OneHotEncoder__Handcap_Yes',
       'remainder__time_bw_schedule_appointment'], dtype=object)

In [324]:
# Create pipelines for numeric and categorical features
numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median", add_indicator=False)),
])

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="most_frequent", add_indicator=False)),
    ('encoder', OneHotEncoder())
])

# Combine numeric and categorical transformations in a ColumnTransformer
preprocessor = ColumnTransformer([
    ('impute_num', numeric_pipeline, ['Age']),
    ('impute_cat', categorical_pipeline, ['SocialWelfare', 'Diabetes', 'Sex', 'Alcoholism', 'Hipertension', 'SMS_received', 'Handcap'])
])
preprocessor.fit_transform(X_df_test)
preprocessor.get_feature_names_out()

array(['impute_num__Age', 'impute_cat__SocialWelfare_No',
       'impute_cat__SocialWelfare_Yes', 'impute_cat__Diabetes_No',
       'impute_cat__Diabetes_Yes', 'impute_cat__Sex_F',
       'impute_cat__Sex_M', 'impute_cat__Alcoholism_No',
       'impute_cat__Alcoholism_Yes', 'impute_cat__Hipertension_No',
       'impute_cat__Hipertension_Yes', 'impute_cat__SMS_received_No',
       'impute_cat__SMS_received_Yes', 'impute_cat__Handcap_No',
       'impute_cat__Handcap_Yes'], dtype=object)

In [ ]:
# occ_test = X_df_test.groupby('Sex').size().reset_index()
# print(occ_test)

In [ ]:
ohe = OneHotEncoder(sparse_output=False, drop='first')

ct = ColumnTransformer(
    [('OneHotEncoder', ohe, ['SocialWelfare', 'Diabetes', 'Sex', 'Alcoholism', 'Hipertension', 'SMS_received', 'Handcap'])],
    remainder='passthrough'
)

ct.fit_transform(X_df_test)

ct.get_feature_names_out()

In [ ]:
y_pred_test = pipe.predict(X_df_test)

### 9. Model Interpretation


### 10. Fine-Tuning and Optimization